
# 🫀 ECG Digitization — Grid Detection & Waveform Extraction (Work in Progress)

This notebook explores the **digitization of ECG paper images** from the *PhysioNet - Digitization of ECG Images* competition.  
The aim is to extract ECG waveforms as numerical time-series data through image preprocessing, grid detection, and contour extraction.

---

### 📘 **Notebook Outline**
1. **Overview & Data Setup**
2. **ECG Image Visualization**
3. **Grid Detection using Hough Transform**
4. **Waveform Extraction via Thresholding**
5. **Digitization: Pixel → Time-Amplitude Conversion**
6. **CSV Export and Visualization**
7. **Next Steps (Rotation, Scale, Multi-lead)**

---

### 💡 **Motivation**
ECG images often exist as scanned printouts, making them hard to analyze digitally.  
By detecting the grid and tracing the waveform line, we can reconstruct time-series ECG data for further analysis or ML modeling.

> *Work in progress — shared for community feedback and educational purposes.*


# PhysioNet - Digitization of ECG Images 🫀
### Starter Notebook (EDA + Grid & Waveform Detection)

本ノートブックは PhysioNet の ECG 画像を対象に、波形データ化（デジタイズ）のための前処理テンプレートです。

**内容:**
1. データ構造の確認と可視化
2. ECG画像の表示
3. グリッド線の検出 (OpenCV Hough変換)
4. 波形抽出 (二値化 + 輪郭抽出)
5. ピクセル座標をデジタル波形に変換
6. CSV出力

---

## 1️⃣ パス設定と基本情報

In [ ]:

import os, cv2, numpy as np, pandas as pd, matplotlib.pyplot as plt

# Kaggle 上のデータ構造に合わせる
BASE_PATH = "/kaggle/input/physionet-ecg-image-digitization"
TRAIN_DIR = f"{BASE_PATH}/train"
TEST_DIR = f"{BASE_PATH}/test"

# サンプルIDを選択
sample_id = "1006427285"
img_name = f"{sample_id}-0001.png"
img_path = f"{TRAIN_DIR}/{sample_id}/{img_name}"
print("Image path:", img_path)

print("Image path2:","/kaggle/input/physionet-ecg-image-digitization/train/1006427285/1006427285-0001.png")


## 2️⃣ ECG画像を読み込み・可視化

In [ ]:

# 画像の読み込み
img = cv2.imread(img_path)
##print("img:",np.array(img))
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

plt.figure(figsize=(12,6))
plt.imshow(gray, cmap='gray')
plt.title(f"Sample ECG: {img_name}")
plt.axis('off')
plt.show()


## 3️⃣ グリッド線の検出（時間軸と振幅軸の目安）

In [ ]:

blur = cv2.GaussianBlur(gray, (3,3), 0)
edges = cv2.Canny(blur, 50, 150, apertureSize=3)

lines = cv2.HoughLines(edges, 1, np.pi / 180, 400)
grid = img.copy()

if lines is not None:
    for rho, theta in lines[:,0]:
        a = np.cos(theta); b = np.sin(theta)
        x0 = a * rho; y0 = b * rho
        x1 = int(x0 + 1000 * (-b)); y1 = int(y0 + 1000 * (a))
        x2 = int(x0 - 1000 * (-b)); y2 = int(y0 - 1000 * (a))
        cv2.line(grid, (x1, y1), (x2, y2), (0,255,0), 1)

plt.figure(figsize=(12,6))
plt.imshow(cv2.cvtColor(grid, cv2.COLOR_BGR2RGB))
plt.title("Detected Grid Lines (ECG background)")
plt.axis('off')
plt.show()


## 4️⃣ 波形抽出（二値化＋輪郭検出）

In [ ]:
# =====================================================
# ① グリッド除去 + ② モルフォロジ + ③ 細線化 自動処理
# =====================================================
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.morphology import skeletonize

def preprocess_ecg_image(img_gray, grid_kernel=30, show=False):
    """
    ECG画像（グレースケール）からグリッド除去＋波形強調を行う。
    grid_kernel: グリッド線の周期（px単位、25mm/sなら30前後）
    """
    # --- ① 背景のグリッド線除去 ---
    # 横線・縦線をそれぞれ削除
    horizontal = cv2.morphologyEx(img_gray, cv2.MORPH_OPEN,
                                  cv2.getStructuringElement(cv2.MORPH_RECT, (grid_kernel, 1)))
    vertical = cv2.morphologyEx(img_gray, cv2.MORPH_OPEN,
                                cv2.getStructuringElement(cv2.MORPH_RECT, (1, grid_kernel)))
    grid = cv2.add(horizontal, vertical)
    no_grid = cv2.subtract(img_gray, grid)
    
    # --- ② 二値化 + モルフォロジ（ノイズ除去＋線補強） ---
    _, binary = cv2.threshold(no_grid, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    kernel = np.ones((3, 3), np.uint8)
    morph = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel, iterations=2)
    morph = cv2.morphologyEx(morph, cv2.MORPH_OPEN, kernel, iterations=1)

    # --- ③ 細線化（1px幅の線に） ---
    # scikit-image の skeletonizeを使用（入力は0,1のfloat画像）
    thin = skeletonize((morph > 0).astype(np.uint8)).astype(np.uint8) * 255

    if show:
        plt.figure(figsize=(14,4))
        plt.subplot(1,4,1); plt.imshow(img_gray, cmap='gray'); plt.title("Gray Input"); plt.axis('off')
        plt.subplot(1,4,2); plt.imshow(no_grid, cmap='gray'); plt.title("No Grid"); plt.axis('off')
        plt.subplot(1,4,3); plt.imshow(morph, cmap='gray'); plt.title("Morph"); plt.axis('off')
        plt.subplot(1,4,4); plt.imshow(thin, cmap='gray'); plt.title("Skeletonized"); plt.axis('off')
        plt.show()

    return thin

# --- テスト例 ---
# img_gray = cv2.imread('/kaggle/input/physionet-ecg-image-digitization/train/1006867983/1006867983-0001.png', cv2.IMREAD_GRAYSCALE)
# processed = preprocess_ecg_image(img_gray, grid_kernel=30, show=True)


In [ ]:
gray_u8 = thin.copy()
if gray_u8.dtype != np.uint8:
    gray_u8 = (np.clip(gray_u8, 0, 255)).astype(np.uint8)

# 1) 白い“細い線”を強調（Top-hat）
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (7,7))
tophat = cv2.morphologyEx(gray_u8, cv2.MORPH_TOPHAT, kernel)

# --- Before thresholding ---
b, g, r = cv2.split(img)
gray_wave = cv2.subtract(255, b)  # 反転して黒波形を強調
plt.imshow(gray_wave, cmap='gray')
plt.title("Enhanced waveform channel")
plt.axis('off')
plt.show()

# Otsu’s binarization
blur = cv2.GaussianBlur(gray_wave, (5,5), 0)
_, binary = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

plt.imshow(binary, cmap='gray')
plt.title("Otsu Threshold Result")
plt.axis('off')
plt.show()

kernel = np.ones((3,3), np.uint8)
binary_closed = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel, iterations=2)

plt.imshow(binary_closed, cmap='gray')
plt.title("After Morphological Closing")
plt.axis('off')
plt.show()

contours, _ = cv2.findContours(binary_closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
wave = np.zeros_like(binary_closed)
cv2.drawContours(wave, contours, -1, 255, 1)

plt.imshow(wave, cmap='gray')
plt.title("Improved Waveform Contours")
plt.axis('off')
plt.show()



# 2) “非反転”の適応二値化（白=波形）
bin_wave = cv2.adaptiveThreshold(
    tophat, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 55, -5
)

# 3) グリッド抑圧（水平/垂直の長いカーネルで開演算して引き算）
hker = cv2.getStructuringElement(cv2.MORPH_RECT, (45,1))
vker = cv2.getStructuringElement(cv2.MORPH_RECT, (1,45))
grid_h = cv2.morphologyEx(bin_wave, cv2.MORPH_OPEN, hker)
grid_v = cv2.morphologyEx(bin_wave, cv2.MORPH_OPEN, vker)
grid = cv2.bitwise_or(grid_h, grid_v)

bin_wo_grid = cv2.bitwise_and(bin_wave, cv2.bitwise_not(grid))

# 4) 輪郭検出（外枠だけに偏らないよう RETR_LIST）
contours, _ = cv2.findContours(bin_wo_grid, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)

# 5) 描画（ゼロ初期化の8bitキャンバスに白で描画）
wave = np.zeros_like(gray_u8)             # ← 同形状・同dtype(uint8)のゼロ配列
cv2.drawContours(wave, contours, -1, 255, 1)

plt.figure(figsize=(12,6))
plt.imshow(binary, cmap='gray')
plt.title("Binary ECG Trace")
plt.axis('off')
plt.show()

plt.figure(figsize=(12,6))
plt.imshow(wave, cmap='gray')
plt.title("Extracted ECG Line Contours")
plt.axis('off')
plt.show()

In [ ]:
# --- Step 1. Morphological closing (with larger kernel) ---
kernel = np.ones((5,5), np.uint8)
binary_closed = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel, iterations=2)

# --- Step 2. Ensure binary is uint8 & white-on-black ---
binary_inv = cv2.bitwise_not(binary_closed)
binary_inv = np.uint8(binary_inv)

plt.imshow(binary_inv, cmap='gray')
plt.title("Binary (Inverted for Contour Detection)")
plt.axis('off')
plt.show()

# --- Step 3. Detect contours ---
contours, _ = cv2.findContours(binary_inv, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

wave = np.zeros_like(binary_inv)
cv2.drawContours(wave, contours, -1, 255, 1)

plt.imshow(wave, cmap='gray')
plt.title("Improved Waveform Contours (Fixed)")
plt.axis('off')
plt.show()

print("Detected contour count:", len(contours))


In [ ]:
wave = np.zeros_like(binary_inv)
cv2.drawContours(wave, contours, -1, 255, 3)   # ← thicknessを3に
plt.imshow(wave, cmap='gray', vmin=0, vmax=255)
plt.title("Contours (Thicker, visible)")
plt.axis('off')
plt.show()


In [ ]:
b, g, r = cv2.split(img)
# 赤グリッド除去 → 黒波形強調
gray_wave = cv2.subtract(255, r)  
blur = cv2.GaussianBlur(gray_wave, (5,5), 0)
_, binary = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

plt.imshow(binary, cmap='gray')
plt.title("Red Grid Removed — Black Wave Enhanced")
plt.axis('off')
plt.show()


In [ ]:
filtered_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > 20]

wave = np.zeros_like(binary_inv)
cv2.drawContours(wave, filtered_contours, -1, 255, 2)
plt.imshow(wave, cmap='gray')
plt.title("Filtered Contours (Waveform only)")
plt.axis('off')
plt.show()

print("After filtering:", len(filtered_contours), "contours kept")


In [ ]:
# --- Grid color suppression (remove reddish pixels) ---
img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
# Hue 0〜10 or 160〜180（赤領域）を除去
mask_red = cv2.inRange(img_hsv, (0,50,50), (10,255,255)) | cv2.inRange(img_hsv, (160,50,50), (180,255,255))
img_no_grid = cv2.inpaint(img, mask_red, 3, cv2.INPAINT_TELEA)

plt.figure(figsize=(12,6))
plt.imshow(cv2.cvtColor(img_no_grid, cv2.COLOR_BGR2RGB))
plt.title("Grid Removed using HSV Mask")
plt.axis('off')
plt.show()


In [ ]:
from skimage.morphology import skeletonize
wave_bin = (wave > 0).astype(np.uint8)
skeleton = skeletonize(wave_bin).astype(np.uint8) * 255

plt.imshow(skeleton, cmap='gray')
plt.title("Skeletonized ECG waveform (1px width)")
plt.axis('off')
plt.show()


#　改善コード

In [ ]:
for img_path in selected_images:
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    thin = preprocess_ecg_image(img, grid_kernel=30)
    # 以降、thin を使って輪郭検出・波形抽出へ


In [ ]:
# --- 1️⃣ 赤系＋ピンク系グリッド除去（範囲拡大） ---
img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

# 赤～ピンクまで広めに指定
mask_red = (
    cv2.inRange(img_hsv, (0, 30, 40), (15, 255, 255)) |  # 赤系
    cv2.inRange(img_hsv, (150, 30, 40), (180, 255, 255))  # ピンク系
)

img_no_grid = cv2.inpaint(img, mask_red, 3, cv2.INPAINT_TELEA)

plt.figure(figsize=(12,6))
plt.imshow(cv2.cvtColor(img_no_grid, cv2.COLOR_BGR2RGB))
plt.title("Grid Fully Removed (Red + Pink HSV Range)")
plt.axis('off')
plt.show()


# --- 2️⃣ 黒波形を強調し、スケルトン化 ---
b, g, r = cv2.split(img_no_grid)
gray_wave = cv2.subtract(255, r)
blur = cv2.GaussianBlur(gray_wave, (5,5), 0)
_, binary = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# 膨張＋収縮で線を整える
kernel = np.ones((3,3), np.uint8)
binary = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel, iterations=2)

# スケルトン化
from skimage.morphology import skeletonize
skeleton = skeletonize((binary > 0).astype(np.uint8)).astype(np.uint8) * 255

plt.figure(figsize=(12,6))
plt.imshow(skeleton, cmap='gray')
plt.title("Final Skeletonized ECG waveform (Grid-Free, 1px)")
plt.axis('off')
plt.show()


In [ ]:
import numpy as np
import pandas as pd

# スケルトン画像から各列の波形座標を抽出
y_positions = []
for x in range(skeleton.shape[1]):
    ys = np.where(skeleton[:, x] > 0)[0]
    if len(ys) > 0:
        y_mean = np.median(ys)
        y_positions.append(y_mean)
    else:
        y_positions.append(np.nan)

# 補間して欠損を埋める
y_series = pd.Series(y_positions).interpolate().fillna(method='bfill').fillna(method='ffill')

# CSVとして保存
pd.DataFrame({"x": np.arange(len(y_series)), "y": y_series}).to_csv("ecg_waveform_extracted.csv", index=False)


In [ ]:
#修正
# 例：Lead II の位置（画像の下1/4あたり）を切り出す
h, w = skeleton.shape
lead_y_start = int(h * 0.70)
lead_y_end = int(h * 0.80)
lead_ii = skeleton[lead_y_start:lead_y_end, :]
plt.imshow(lead_ii, cmap='gray')
plt.title("Extracted Lead II Region")
plt.axis('off')
plt.show()


In [ ]:
y_positions = []
for x in range(lead_ii.shape[1]):
    ys = np.where(lead_ii[:, x] > 0)[0]
    if len(ys) > 0:
        y_positions.append(np.median(ys))
    else:
        y_positions.append(np.nan)

# 欠損補間 & 平滑化
import pandas as pd
import numpy as np
from scipy.signal import savgol_filter

y_series = pd.Series(y_positions).interpolate().fillna(method='bfill').fillna(method='ffill')
y_smooth = savgol_filter(y_series, window_length=31, polyorder=3)  # 平滑化

plt.figure(figsize=(12,4))
plt.plot(y_smooth)
plt.gca().invert_yaxis()
plt.title("Smoothed Single Lead ECG (Lead II)")
plt.xlabel("Time (pixels)")
plt.ylabel("Amplitude (pixels)")
plt.show()


In [ ]:
import numpy as np
import pandas as pd
from scipy.signal import savgol_filter
import matplotlib.pyplot as plt

# === 1. スケルトン波形から1リード領域切り出し ===
h, w = skeleton.shape
lead_y_start = int(h * 0.70)   # 前回より少し狭めに設定
lead_y_end   = int(h * 0.78)
lead_ii = skeleton[lead_y_start:lead_y_end, :]

# === 2. 各列ごとの白画素の中央値（波形） ===
y_positions = []
for x in range(lead_ii.shape[1]):
    ys = np.where(lead_ii[:, x] > 0)[0]
    if len(ys) > 0:
        y_positions.append(np.median(ys))
    else:
        y_positions.append(np.nan)

# 欠損補間
y_series = pd.Series(y_positions).interpolate().fillna(method='bfill').fillna(method='ffill')

# === 3. ベースライン補正（移動平均差し引き） ===
baseline = y_series.rolling(window=200, center=True, min_periods=1).mean()
y_corrected = y_series - baseline

# === 4. 平滑化（適度なwindowとpoly） ===
y_smooth = savgol_filter(y_corrected, window_length=51, polyorder=3)

# === 5. 表示（上下反転で心電図らしく） ===
plt.figure(figsize=(12,4))
plt.plot(-y_smooth, color='steelblue')
plt.title("Lead II — Baseline Corrected & Smoothed ECG Waveform")
plt.xlabel("Time (pixels)")
plt.ylabel("Amplitude (a.u.)")
plt.show()


## 5️⃣ 波形をピクセル座標からデジタル化

In [ ]:
plt.figure(figsize=(12,4))
plt.plot(y_series.values)
plt.gca().invert_yaxis()
plt.title("Digitized ECG waveform")
plt.xlabel("Time (pixels)")
plt.ylabel("Amplitude (pixels)")
plt.show()

In [ ]:

ys = []
for x in range(wave.shape[1]):
    ys_in_col = np.where(wave[:,x] > 0)[0]
    if len(ys_in_col) > 0:
        ys.append(np.mean(ys_in_col))
    else:
        ys.append(np.nan)

ys = np.array(ys)
ys = pd.Series(ys).interpolate().fillna(method='bfill').fillna(method='ffill')

plt.figure(figsize=(12,4))
plt.plot(ys.values)
plt.gca().invert_yaxis()
plt.title("Digitized ECG waveform")
plt.xlabel("Time (pixels)")
plt.ylabel("Amplitude (pixels)")
plt.show()


## 6️⃣ 波形を正規化してCSV保存

In [ ]:

time = np.arange(len(ys)) / 500  # 仮の500Hzサンプリング
amp = (ys - ys.mean()) / ys.std()

df_wave = pd.DataFrame({"time": time, "amplitude": amp})
out_path = "/kaggle/working/sample_waveform.csv"
df_wave.to_csv(out_path, index=False)
print(f"Saved → {out_path}")
df_wave.head()


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import find_peaks, savgol_filter

# === 1️⃣ 自動リード帯検出 ===
# 各行方向の画素数をカウントして最大値の帯域をLead IIと推定
proj = np.sum(skeleton > 0, axis=1)
peaks, _ = find_peaks(proj, distance=100, prominence=100)
lead_index = peaks[1] if len(peaks) > 1 else peaks[0]  # 2番目のピークをLead IIとみなす
band = 30  # 帯域幅 (±px)
lead_ii = skeleton[lead_index-band:lead_index+band, :]

plt.figure(figsize=(10,3))
plt.imshow(lead_ii, cmap='gray')
plt.title("Auto-detected Lead II region")
plt.axis('off')
plt.show()


# === 2️⃣ 列ごとに前列との連続性で最も近い点を採用 ===
y_positions = []
prev_y = None
for x in range(lead_ii.shape[1]):
    ys = np.where(lead_ii[:, x] > 0)[0]
    if len(ys) == 0:
        y_positions.append(np.nan)
        continue
    if prev_y is None:
        y = np.median(ys)
    else:
        y = ys[np.argmin(np.abs(ys - prev_y))]  # 最も近い点を選択
    y_positions.append(y)
    prev_y = y

y_series = pd.Series(y_positions).interpolate().fillna(method='bfill').fillna(method='ffill')

# === 3️⃣ ベースライン補正 ===
baseline = y_series.rolling(window=200, center=True, min_periods=1).median()
y_corrected = y_series - baseline

# === 4️⃣ 外れ値除去（Hampelフィルタ） ===
def hampel(x, k=25, nsigma=3):
    s = pd.Series(x)
    med = s.rolling(k, center=True).median()
    mad = (s - med).abs().rolling(k, center=True).median() * 1.4826
    outlier = (np.abs(s - med) > nsigma * mad)
    s[outlier] = med[outlier]
    return s.to_numpy()

y_denoised = hampel(y_corrected)

# === 5️⃣ 自動平滑化（R–R間隔から窓決定） ===
peaks, _ = find_peaks(-y_denoised, distance=150)
if len(peaks) > 1:
    rr = np.median(np.diff(peaks))
    win = int(max(31, rr // 3) // 2 * 2 + 1)  # 奇数化
else:
    win = 51

y_smooth = savgol_filter(y_denoised, window_length=win, polyorder=3)

# === 6️⃣ 表示 ===
plt.figure(figsize=(12,4))
plt.plot(-y_smooth, color='steelblue')
plt.title(f"Lead II — Auto-corrected & Smoothed ECG (window={win})")
plt.xlabel("Time (pixels)")
plt.ylabel("Amplitude (a.u.)")
plt.show()


In [ ]:
# =====================================================
# ECG Waveform Centerline Extraction & CSV Export
# =====================================================

import numpy as np
import pandas as pd
import cv2
from scipy.signal import find_peaks

def extract_centerline_from_mask(mask, grid_dx
                                 , grid_dy, scale_time=25.0, scale_mv=10.0):
    """
    mask: 2D np.ndarray (0–255), 白線が波形のマスク画像
    grid_dx, grid_dy: グリッド間隔（ピクセル）
    scale_time: mm/s（ECG標準は25mm/s）
    scale_mv: mm/mV（標準は10mm/mV）

    return: DataFrame [time (s), voltage (mV)]
    """
    grid_dx = 22   # 画像の横スケールが広めなら少し大きく
    grid_dy = 18   # 縦スケールが狭いなら少し小さく
    
    h, w = mask.shape
    centerline_y = []
    x_coords = np.arange(w)

    # 列ごとに白ピクセルの平均y座標を求める
    for x in range(w):
        ys = np.where(mask[:, x] > 0)[0]
        if len(ys) > 0:
            center_y = np.mean(ys)
        else:
            center_y = np.nan
        centerline_y.append(center_y)

    centerline_y = np.array(centerline_y)

    # 欠損を補間（線形補間＋前後補完）
    nans = np.isnan(centerline_y)
    if np.any(nans):
        not_nan = np.logical_not(nans)
        centerline_y[nans] = np.interp(np.flatnonzero(nans), np.flatnonzero(not_nan), centerline_y[not_nan])

    # ---- ピクセル → 時間 / 電位 換算 ----
    mm_per_px_x = 1.0 / grid_dx    # 横方向: 1px = (1/grid_dx) mm
    mm_per_px_y = 1.0 / grid_dy    # 縦方向: 同上

    # 時間軸 [秒]
    time_sec = x_coords * mm_per_px_x / scale_time

    # 電位 [mV]
    # y方向は上がマイナスなので、反転して中央基準に
    baseline = np.nanmedian(centerline_y)
    mv = -(centerline_y - baseline) * mm_per_px_y / scale_mv

    df = pd.DataFrame({"time_sec": time_sec, "voltage_mV": mv})
    # yの欠損補完
    df_wave["voltage_mV"] = df_wave["voltage_mV"].interpolate(method="linear")

    return df


# =====================================================
# Example usage
# =====================================================

# 例: wave_mask に波形マスク画像 (0-255) がある場合
# grid_dx, grid_dy は前処理で推定した平均グリッド間隔を指定

df_wave = extract_centerline_from_mask(
    mask,#k=wave_mask,          # waveは白線マスク画像
    #grid_dx=20,         # 例：1mmあたり20px
    #grid_dy=20,
    grid_dx = 22,  # 画像の横スケールが広めなら少し大きく
    grid_dy = 18,   # 縦スケールが狭いなら少し小さく

    
    scale_time=25.0,
    scale_mv=10.0
)

from scipy.signal import savgol_filter

# 波形平滑化を適用（ウィンドウ=9, 多項式=3 は経験的に安定）
df_wave["voltage_mV_smooth"] = savgol_filter(df_wave["voltage_mV"], window_length=9, polyorder=3)

# プロットを差し替え
plt.figure(figsize=(8,3))
plt.plot(df_wave["time_sec"], df_wave["voltage_mV_smooth"], color="red")
plt.title(f"Smoothed ECG Trace: {lead_name}")
plt.xlabel("Time [sec]"); plt.ylabel("Voltage [mV]")
plt.grid(True); plt.tight_layout(); plt.show()

# 保存
lead_name = "V1"  # リード名を適宜変更
save_path = f"./ecg_trace_{lead_name}.csv"
df_wave.to_csv(save_path, index=False)
print(f"Saved: {save_path}")
display(df_wave.head())

# 確認用プロット
plt.figure(figsize=(8, 3))
plt.plot(df_wave["time_sec"], df_wave["voltage_mV"], color="red")
plt.title(f"Extracted ECG Trace ({lead_name})")
plt.xlabel("Time [sec]")
plt.ylabel("Voltage [mV]")
plt.grid(True)
plt.show()


In [ ]:
# =====================================================
# Recursive ECG Digitization Pipeline (subfolder aware)
# =====================================================

import os
import glob
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

# 既存関数 extract_centerline_from_mask() をそのまま使用
# from previous cell import extract_centerline_from_mask

# =====================================================
# 設定
# =====================================================
root_dir = "/kaggle/input/physionet-ecg-image-digitization/train"             # trainフォルダをルートに指定
output_root = "/kaggle/working/ecg_csv_outputs"

os.makedirs(output_root, exist_ok=True)

# サブフォルダも含めて全てのpng/jpgを探索
image_files = sorted(glob.glob(os.path.join(root_dir, "**", "*.png"), recursive=True) +
                     glob.glob(os.path.join(root_dir, "**", "*.jpg"), recursive=True))

print(f"Found {len(image_files)} ECG images under {root_dir}.")

# =====================================================
# パラメータ設定
# =====================================================
grid_dx = 20
grid_dy = 20
scale_time = 25.0
scale_mv = 10.0



# =====================================================
# メインループ
# =====================================================
for path in tqdm(image_files, desc="Processing ECG images"):

    # ---- 画像読み込み ----
    img = cv2.imread(path)
    if img is None:
        print(f"⚠️ Cannot open: {path}")
        continue
    # ---- 画像読込 ----
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    thin = preprocess_ecg_image(gray, grid_kernel=30)


    # ---- 波形抽出（簡易二値化） ----
    blur = cv2.GaussianBlur(gray, (3, 3), 0)
    _, binary = cv2.threshold(blur, 180, 255, cv2.THRESH_BINARY_INV)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
    binary = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)

    # ---- 波形中心線抽出 ----
    df_wave = extract_centerline_from_mask(
        mask=binary,
        grid_dx=grid_dx,
        grid_dy=grid_dy,
        scale_time=scale_time,
        scale_mv=scale_mv
    )

    # ---- 出力先パスを決定（フォルダ階層を再現） ----
    rel_path = os.path.relpath(path, root_dir)
    rel_dir = os.path.dirname(rel_path)
    out_dir = os.path.join(output_root, rel_dir)
    os.makedirs(out_dir, exist_ok=True)

    base_name = os.path.splitext(os.path.basename(path))[0]
    out_csv = os.path.join(out_dir, f"{base_name}.csv")
    df_wave.to_csv(out_csv, index=False)

    # ---- サンプル画像を数件だけプロット ----
    if np.random.rand() < 0.02:  # 全体の2%だけ可視化（多すぎ防止）
        plt.figure(figsize=(8, 3))
        plt.plot(df_wave["time_sec"], df_wave["voltage_mV"], color="red")
        plt.title(f"Extracted: {rel_path}")
        plt.xlabel("Time [sec]")
        plt.ylabel("Voltage [mV]")
        plt.grid(True)
        plt.tight_layout()
        plt.show()

print("✅ Completed ECG extraction for all subfolders.")


In [ ]:
# =====================================================
# Multi-File ECG Digitization Pipeline
# =====================================================

import os
import glob
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

# --- 既に定義済みの関数を使う想定 ---
# from previous cell:
#   extract_centerline_from_mask(mask, grid_dx, grid_dy, scale_time, scale_mv)

# =====================================================
# 設定
# =====================================================
input_dir = "/kaggle/input/physionet-ecg-image-digitization/train/1006867983/"       # ECG画像を入れたフォルダを指定
output_dir = "./ecg_csv_outputs"  # CSV出力先

os.makedirs(output_dir, exist_ok=True)

# 画像ファイルの一覧取得（png, jpgどちらも対応）
image_files = sorted(glob.glob(os.path.join(input_dir, "*.png")) +
                     glob.glob(os.path.join(input_dir, "*.jpg")))

print(f"Found {len(image_files)} ECG images.")

# =====================================================
# パラメータ設定
# =====================================================
#grid_dx = 20   # 1mmあたりのピクセル（横）
#grid_dy = 20   # 1mmあたりのピクセル（縦）
grid_dx = 22   # 画像の横スケールが広めなら少し大きく
grid_dy = 18   # 縦スケールが狭いなら少し小さく
scale_time = 25.0  # 25mm/s
scale_mv = 10.0    # 10mm/mV




# =====================================================
# メイン処理ループ
# =====================================================
for path in tqdm(image_files, desc="Processing ECG images"):

    # ---- 画像読込 ----
    img = cv2.imread(path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    thin = preprocess_ecg_image(gray, grid_kernel=22)
    
    blur = cv2.GaussianBlur(thin, (3, 3), 0)
    _, binary = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
    binary = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)
    binary = cv2.bitwise_not(binary)
    """
    thin = preprocess_ecg_image(gray, grid_kernel=30)

    # ---- 波形マスク生成（簡易二値化）----
    # 背景のグリッド除去と波形線抽出
    blur = cv2.GaussianBlur(thin, (3, 3), 0)
    _, binary = cv2.threshold(blur, 180, 255, cv2.THRESH_BINARY_INV)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
    binary = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)
    """
    # ---- 波形中心線抽出 ----
    df_wave = extract_centerline_from_mask(
        mask=binary,
        grid_dx=grid_dx,
        grid_dy=grid_dy,
        scale_time=scale_time,
        scale_mv=scale_mv
    )

    
    from scipy.signal import savgol_filter
    from scipy.ndimage import median_filter
    
    # --- 波形補間 + 平滑化 ---
    # 欠損値補完
    df_wave["voltage_mV"] = df_wave["voltage_mV"].interpolate(method="linear")
    
    # 外れ値除去（3点メディアン）
    df_wave["voltage_mV"] = median_filter(df_wave["voltage_mV"], size=3)
    
    # サヴィツキー・ゴレイで滑らか化
    df_wave["voltage_mV_smooth"] = savgol_filter(df_wave["voltage_mV"], window_length=11, polyorder=3)
    
    # ベースライン補正（平均値をゼロに）
    df_wave["voltage_mV_smooth"] -= df_wave["voltage_mV_smooth"].mean()
    
    # --- プロット ---
    plt.figure(figsize=(8,3))
    plt.plot(df_wave["time_sec"], df_wave["voltage_mV_smooth"], color="red", lw=1)
    plt.title(f"Smoothed ECG Trace: {lead_name}")
    plt.xlabel("Time [sec]"); plt.ylabel("Voltage [mV]")
    plt.grid(True); plt.tight_layout(); plt.show()




    
    # ---- ファイル名設定 ----
    base = os.path.basename(path)
    lead_name = os.path.splitext(base)[0]
    save_csv = os.path.join(output_dir, f"{lead_name}.csv")
    df_wave.to_csv(save_csv, index=False)

    # ---- 確認プロット ----
    plt.figure(figsize=(8, 3))
    plt.plot(df_wave["time_sec"], df_wave["voltage_mV"], color="red")
    plt.title(f"Extracted ECG Trace: {lead_name}")
    plt.xlabel("Time [sec]")
    plt.ylabel("Voltage [mV]")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

print("✅ Completed ECG extraction for all files.")


In [ ]:
# =====================================================
# Combine 12-lead ECG CSVs into one aligned record ver2
# =====================================================

import os
import pandas as pd
import numpy as np
import glob
from scipy.signal import correlate

def align_and_combine_leads(csv_folder, max_shift=0.2, sample_rate=500):
    """
    指定フォルダ内の12リードCSVを1つのDataFrameに整形
    （時間軸・位相をそろえて統合）
    
    Parameters:
      csv_folder: 各リードのCSVが入ったフォルダ
      max_shift: 時間シフトの許容範囲 [秒]
      sample_rate: サンプリング周波数 [Hz] （25mm/s → 約500Hz）
    Returns:
      pd.DataFrame: time列＋12リード列
    """
    csv_files = sorted(glob.glob(os.path.join(csv_folder, "*.csv")))
    if not csv_files:
        print(f"⚠️ No CSV files in {csv_folder}")
        return None

    leads = {}
    base_time = None

    for f in csv_files:
        lead_name = os.path.splitext(os.path.basename(f))[0].split("-")[-1]  # ファイル末尾から誘導名推定
        df = pd.read_csv(f)
        if base_time is None:
            base_time = df["time_sec"].values
        leads[lead_name] = df["voltage_mV"].values

    # 長さ合わせ
    min_len = min(len(v) for v in leads.values())
    for k in leads:
        leads[k] = leads[k][:min_len]
    base_time = base_time[:min_len]

    # 時間シフト補正（II誘導を基準）
    ref = leads.get("0001", list(leads.values())[0])  # 最初のリードを基準に
    aligned = {}
    for name, sig in leads.items():
        corr = correlate(ref - np.mean(ref), sig - np.mean(sig), mode="full")
        lag = np.argmax(corr) - len(sig) + 1
        lag_time = lag / sample_rate
        if abs(lag_time) > max_shift:
            lag = 0  # 大きすぎるシフトは無視
        aligned[name] = np.roll(sig, -lag)

    # DataFrameへ統合
    df_combined = pd.DataFrame({"time_sec": base_time})
    for name, sig in aligned.items():
        df_combined[name] = sig

    return df_combined

# =====================================================
# Example: Combine one patient's leads
# =====================================================
record_folder = "./ecg_csv_outputs/1006867983"  # 対象フォルダを指定
df_12lead = align_and_combine_leads(record_folder)

if df_12lead is not None:
    display(df_12lead.head())
    plt.figure(figsize=(10,6))
    for col in df_12lead.columns[1:]:
        plt.plot(df_12lead["time_sec"], df_12lead[col] + 2*list(df_12lead.columns[1:]).index(col), label=col)
    plt.title(f"12-lead ECG Combined View ({os.path.basename(record_folder)})")
    plt.xlabel("Time [sec]")
    plt.ylabel("Relative Voltage [mV]")
    plt.legend(ncol=4, fontsize=8)
    plt.grid(True)
    plt.show()


In [ ]:
# =====================================================
# SNR Local Evaluation Function for PhysioNet ECG
# =====================================================

import numpy as np
from scipy.signal import correlate

def align_signals(pred, truth, sample_rate=500, max_shift=0.2):
    """
    時間軸シフトを補正して2つの信号を整列。
    最大±max_shift秒の範囲で相互相関を最大化。
    """
    max_lag = int(max_shift * sample_rate)
    corr = correlate(truth - np.mean(truth), pred - np.mean(pred), mode="full")
    lags = np.arange(-len(pred)+1, len(truth))
    valid_idx = (lags >= -max_lag) & (lags <= max_lag)
    lag = lags[valid_idx][np.argmax(corr[valid_idx])]
    aligned_pred = np.roll(pred, lag)
    return aligned_pred, lag / sample_rate


def compute_snr(pred, truth):
    """
    縦オフセットを補正し、信号対雑音比（SNR, dB）を算出。
    """
    pred = pred - np.mean(pred)
    truth = truth - np.mean(truth)
    noise = truth - pred
    snr = 10 * np.log10(np.sum(truth**2) / np.sum(noise**2))
    return snr


def evaluate_ecg_record(pred_df, truth_df, sample_rate=500):
    """
    各リードごとにSNRを計算し、平均を返す。
    """
    leads = [c for c in truth_df.columns if c != "time_sec" and c in pred_df.columns]
    snr_list = []

    for lead in leads:
        pred = pred_df[lead].values
        truth = truth_df[lead].values

        # 長さ調整
        min_len = min(len(pred), len(truth))
        pred, truth = pred[:min_len], truth[:min_len]

        # 整列・補正
        aligned_pred, shift = align_signals(pred, truth, sample_rate=sample_rate)
        snr = compute_snr(aligned_pred, truth)
        snr_list.append(snr)

        print(f"{lead:>5s} | Shift: {shift:+.4f}s | SNR: {snr:.2f} dB")

    avg_snr = np.mean(snr_list)
    print(f"\n✅ Mean SNR across leads: {avg_snr:.2f} dB")
    return avg_snr


In [ ]:
import numpy as np
from scipy.signal import correlate

def _get_time_and_leads(df):
    time_col = "time_sec" if "time_sec" in df.columns else ("time" if "time" in df.columns else None)
    if time_col is None:
        raise ValueError("時間列(time/time_sec)が見つかりません。")
    leads = [c for c in df.columns if c != time_col]
    return time_col, leads

def _align_by_xcorr(pred, truth, sample_rate, max_shift=0.2):
    max_lag = int(max_shift * sample_rate)
    corr = correlate(truth - np.mean(truth), pred - np.mean(pred), mode="full")
    lags = np.arange(-len(pred)+1, len(truth))
    sel = (lags >= -max_lag) & (lags <= max_lag)
    lag = lags[sel][np.argmax(corr[sel])] if np.any(sel) else 0
    return np.roll(pred, lag), lag / sample_rate

def evaluate_ecg_record_v2(pred_df, truth_df):
    # 時間列取得
    p_time, p_leads = _get_time_and_leads(pred_df)
    t_time, t_leads = _get_time_and_leads(truth_df)
    # 共通リード
    common = [l for l in p_leads if l in t_leads]
    if not common:
        raise ValueError(f"共通するリードがありません。\n pred:{p_leads}\n truth:{t_leads}")
    # サンプルレート推定（時間差の中央値から）
    dt_pred = np.median(np.diff(pred_df[p_time].values))
    sample_rate = int(round(1.0 / dt_pred)) if dt_pred > 0 else 500

    snrs = []
    for lead in common:
        p = pred_df[lead].values
        t = truth_df[lead].values
        n = min(len(p), len(t))
        p, t = p[:n], t[:n]
        p_aligned, shift = _align_by_xcorr(p, t, sample_rate=sample_rate, max_shift=0.2)
        # 縦オフセット除去
        p_aligned = p_aligned - np.mean(p_aligned)
        t = t - np.mean(t)
        noise = t - p_aligned
        if np.sum(noise**2) == 0:
            snr = np.inf
        else:
            snr = 10*np.log10(np.sum(t**2)/np.sum(noise**2))
        print(f"{lead:>5s} | shift={shift:+.4f}s | SNR={snr:.2f} dB")
        snrs.append(snr)
    mean_snr = float(np.mean(snrs))
    print(f"\n✅ Mean SNR across leads: {mean_snr:.2f} dB (n={len(snrs)})")
    return mean_snr

# 実行
mean_snr = evaluate_ecg_record_v2(pred_df, truth_df)


In [ ]:
import numpy as np
from scipy.signal import correlate

def _infer_fs_from_time(df):
    tcol = "time_sec" if "time_sec" in df.columns else ("time" if "time" in df.columns else None)
    if tcol is None:
        return 500  # フォールバック
    dt = np.median(np.diff(df[tcol].values))
    return int(round(1.0/dt)) if dt and np.isfinite(dt) and dt>0 else 500

def _align_xcorr_valid(pred, truth, fs, max_shift_s=0.2):
    """NaNを除外しつつ相互相関でシフト最適化（±max_shift_s）"""
    max_lag = int(max_shift_s * fs)
    # NaN除外
    mask = np.isfinite(pred) & np.isfinite(truth)
    if mask.sum() < 10:  # データ少なすぎ
        return pred, 0.0
    p = pred[mask] - np.nanmean(pred[mask])
    q = truth[mask] - np.nanmean(truth[mask])
    # 相関
    corr = correlate(q, p, mode="full")
    lags = np.arange(-len(p)+1, len(q))
    ok = (lags >= -max_lag) & (lags <= max_lag)
    lag = lags[ok][np.argmax(corr[ok])] if ok.any() else 0
    # シフト適用（全長に対して）
    pred_shift = np.roll(pred, lag)
    return pred_shift, lag / fs

def evaluate_ecg_record_robust(pred_df, truth_df, fs=None, max_shift_s=0.2, verbose=True):
    """
    各リードで truth の「有限値が連続している先頭区間」だけを使い、
    pred を同じ長さに切り出してから整列＆SNR計算。
    """
    if fs is None:
        fs = _infer_fs_from_time(pred_df)

    p_time = "time_sec" if "time_sec" in pred_df.columns else "time"
    t_time = "time_sec" if "time_sec" in truth_df.columns else "time"

    pred_leads = [c for c in pred_df.columns if c != p_time]
    truth_leads = [c for c in truth_df.columns if c != t_time]
    common = [l for l in pred_leads if l in truth_leads]
    if not common:
        raise ValueError(f"共通リードがありません: pred={pred_leads}, truth={truth_leads}")

    snr_list = []
    for lead in common:
        p = pred_df[lead].values.astype(float)
        t = truth_df[lead].values.astype(float)

        # ---- truth の「先頭から連続する有限区間」のみ使う ----
        finite_truth = np.isfinite(t)
        if not finite_truth.any():
            if verbose: print(f"{lead:>5s} | no finite truth -> skip")
            continue
        # 先頭からの連続区間長
        valid_len = np.argmax(~finite_truth) if (~finite_truth).any() else len(t)
        t_seg = t[:valid_len]
        p_seg = p[:valid_len] if len(p) >= valid_len else p  # predが短ければそのまま

        # 長さ合わせ
        n = min(len(p_seg), len(t_seg))
        if n < 10:
            if verbose: print(f"{lead:>5s} | too short overlap -> skip")
            continue
        p_seg = p_seg[:n]
        t_seg = t_seg[:n]

        # 整列（NaNを考慮した相関）
        p_aligned, shift = _align_xcorr_valid(p_seg, t_seg, fs, max_shift_s=max_shift_s)

        # オフセット除去 & NaN除去
        mask = np.isfinite(p_aligned) & np.isfinite(t_seg)
        if mask.sum() < 10:
            if verbose: print(f"{lead:>5s} | not enough finite -> skip")
            continue
        p0 = p_aligned[mask] - np.mean(p_aligned[mask])
        t0 = t_seg[mask] - np.mean(t_seg[mask])

        noise = t0 - p0
        denom = np.sum(noise**2)
        if denom == 0:
            snr = np.inf
        else:
            snr = 10*np.log10(np.sum(t0**2) / denom)

        if verbose:
            print(f"{lead:>5s} | len={len(t0):4d} | shift={shift:+.4f}s | SNR={snr:.2f} dB")
        snr_list.append(snr)

    if len(snr_list)==0:
        if verbose: print("⚠️ 有効なリードがありません（NaNのみ）。")
        return np.nan

    mean_snr = float(np.nanmean(snr_list))
    if verbose:
        print(f"\n✅ Mean SNR across leads: {mean_snr:.2f} dB (n={len(snr_list)})")
    return mean_snr


In [ ]:
truth_df

In [ ]:
# =====================================================
# Example Usage
# =====================================================

# truth_df:  本当のECG時系列（PhysioNet提供のground truth）
# pred_df:   Keiさんの抽出した波形（df_12leadなど）

# 例: Check 同じ/kaggle/input/physionet-ecg-image-digitization/train/1006867983フォルダ内に正解ファイルがある場合
truth_path = "/kaggle/input/physionet-ecg-image-digitization/train/1006867983/1006867983.csv"
truth_df = pd.read_csv(truth_path)

# Keiさんの推定結果（すでに作成済み）
pred_df = df_12lead.copy()

# 典型的なマッピング例（必要に応じて調整）
alias_to_num = {
    "I":"0001","II":"0003","III":"0004",
    "aVR":"0005","aVL":"0006","aVF":"0009",
    "V1":"0010","V2":"0011","V3":"0012",
    "V4":"0013","V5":"0014","V6":"0015"  # 末尾の番号は実データに合わせて変更する
}

def normalize_lead_columns(df):
    cols = list(df.columns)
    # 時間列
    time_col = "time_sec" if "time_sec" in cols else ("time" if "time" in cols else None)
    # リード列の名前を数字系に寄せる
    new = {}
    for c in cols:
        if c == time_col: 
            new[c] = "time_sec"
        elif c in alias_to_num:
            new[c] = alias_to_num[c]
        else:
            new[c] = c  # すでに 0001 等ならそのまま
    df = df.rename(columns=new)
    return df

truth_df = normalize_lead_columns(truth_df)
pred_df  = normalize_lead_columns(df_12lead.copy())

print("truth_df (norm) cols:", [c for c in truth_df.columns if c!="time_sec"][:15])
print("pred_df  (norm) cols:", [c for c in pred_df.columns  if c!="time_sec"][:15])

#mean_snr = evaluate_ecg_record(pred_df, truth_df)


In [ ]:
# 欠損の多いtruth_dfをリードごとに分割
truth_signals = {}
for col in truth_df.columns:
    if col.startswith("000") or col in ["I", "II", "III", "aVR", "aVL", "aVF", "V1","V2","V3","V4","V5","V6"]:
        sig = truth_df[col].dropna().values
        truth_signals[col] = sig
        print(f"{col}: {len(sig)} samples")


## ステップ2: 仮の時間列を作る

In [ ]:
sample_rate = 500
truth_signals_time = {
    lead: np.arange(len(sig)) / sample_rate for lead, sig in truth_signals.items()
}


## ステップ3: 時間列つき DataFrame 化

In [ ]:
import pandas as pd

truth_dict = {}
max_len = max(len(sig) for sig in truth_signals.values())
time_sec = np.arange(max_len) / sample_rate
truth_dict["time_sec"] = time_sec

for lead, sig in truth_signals.items():
    pad = np.full(max_len, np.nan)
    pad[:len(sig)] = sig
    truth_dict[lead] = pad

truth_df_fixed = pd.DataFrame(truth_dict)
print(truth_df_fixed.shape)
display(truth_df_fixed.head())


## ステップ4: 評価実行

In [ ]:
# pred_df はすでに作成済み（12誘導整形したもの）
# truth_df_fixed は「仮の time_sec を付けたもの」

mean_snr = evaluate_ecg_record_robust(pred_df, truth_df_fixed)



In [ ]:
import os, glob, pandas as pd

rec_id = "1006867983"  # 比較したいレコードID
folder = f"/kaggle/input/physionet-ecg-image-digitization/train/{rec_id}"
print("CSV in folder:", glob.glob(os.path.join(folder, "*.csv")))

truth_path = glob.glob(os.path.join(folder, "*.csv"))[0]  # 1つだけ想定
truth_df = pd.read_csv(truth_path)
print("truth_df columns:", list(truth_df.columns)[:20])
display(truth_df.head(3))



---
## 🚀 Next Improvements
- [ ] **Auto-rotation correction** — use average Hough angle to align grid horizontally  
- [ ] **Grid interval estimation** — derive physical scale (ms, mV) from grid spacing  
- [ ] **Multi-lead segmentation** — detect and separate lead regions (I, II, V1–V6)  
- [ ] **Waveform denoising and R-peak detection** — prepare for signal reconstruction  

---

🧩 *Author: Keiko Itano (Lumo.Kei)*  
Kaggle Notebook prepared for the PhysioNet Digitization of ECG Images challenge.  
Feedback and collaboration are welcome!
